In [1]:
suppressPackageStartupMessages(library(cicero))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(parallel))

In [2]:
clus <- makeCluster(32)
#clusterExport(clus)

In [3]:
wd = '/nfs/lab/jnewsome/pbmc/cicero_v4/'
setwd(wd)

In [4]:
zz   = gzfile('/nfs/lab/jnewsome/pbmc/cicero_v4/lft_mtx_notSplit/pbmc1_15.b.merged_peaks.long_fmt.mtx.gz','rt')   ### long for matrix peak x barcode
data = read.table(zz,header=T,  sep="\t")

In [5]:
# ### convert to sparse matrix peak x barcode
# sc.data <- with(data, sparseMatrix(i=as.numeric(peak), j=as.numeric(barcode), 
#                                    x=value, dimnames=list(levels(peak), levels(barcode))))
# rownames(sc.data) <- paste0('chr', gsub('-','_', gsub(':','_',rownames(sc.data))))


In [6]:
sc.umap <- read.table( '/nfs/lab/jnewsome/pbmc/split_cicero_2/splitMeta/cellSplit/pbmc1-15.b.metadata.txt', sep='\t', header=T, row.names=1) ### map barcode to cluster 

In [20]:
cluster_peaks = read.table("/nfs/lab/projects/pbmc_snATAC/pipeline/snATAC/combined_files/pbmc1to15/peak_call/broad_pop/annotated_peak_bed/b.merged_peaks.anno.bed") ### peaks called in each cluster to convert as wide format


In [21]:
cluster_peaks

V1,V2,V3,V4
<fct>,<int>,<int>,<fct>
chr1,9919,10556,b
chr1,29215,29505,b
chr1,237612,237913,b
chr1,521439,521711,b
chr1,565193,565802,b
chr1,569239,569908,b
chr1,713683,714587,b
chr1,740147,740447,b
chr1,752536,752914,b


In [23]:


mat   = str_split_fixed( cluster_peaks[,4], "\\,", length(unique(sc.umap$broad_populations)))
mat2  = t( apply( mat, 1, function(x) as.numeric(unique(sc.umap$broad_populations) %in% x)))


In [28]:
mat2

1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1


In [30]:
peaks

chr1_9919_10556,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1


In [29]:
cluster_peaks[,1]

[1] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [13] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [25] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [37] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [49] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [61] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [73] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [85] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
   [97] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [109] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [121] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [133] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [145] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [157] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [169] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [181] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [193] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [205] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [217] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [229] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [241] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [253] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [265] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [277] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [289] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [301] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [313] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [325] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [337] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [349] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [361] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [373] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [385] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [397] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [409] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [421] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [433] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [445] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [457] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [469] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [481] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [493] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [505] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [517] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [529] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [541] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [553] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [565] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [577] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [589] chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1  chr1 
  [6

In [44]:
# peaks2 = cbind(paste(cluster_peaks[,1], cluster_peaks[,2],  cluster_peaks[,3], sep="_"), 1)

In [45]:
# peaks2

In [34]:
# peaks = cbind(paste(cluster_peaks[,1], cluster_peaks[,2],  cluster_peaks[,3], sep="_"), mat2)
# colnames(peaks) = c("peak", unique(as.character(sc.umap$broad_populations)))

In [36]:
colnames(peaks2) = c("peak", unique(as.character(sc.umap$broad_populations)))

In [40]:
peaks <- peaks2

In [41]:
# clusters <- unique(sc.umap$cluster)

clusters <- c('b')

In [ ]:
# for (cluster in clusters) {
#     sc.umap.subset <- sc.umap[sc.umap[['broad_populations']]==cluster,]
#     sc.data.subset <- sc.data[ rownames(sc.data) %in% peaks[peaks[, cluster]>0,'peak'], ]
#     sc.data.subset <- sc.data.subset[ ,colnames(sc.data.subset) %in% rownames(sc.umap.subset)] > 0
    
#     cellinfo <-data.frame(cells=colnames(sc.data.subset))
#     row.names(cellinfo) <- cellinfo$cells
#     dhsinfo <- data.frame(site_name=rownames(sc.data.subset))
#     row.names(dhsinfo) <- dhsinfo$site_name
#     dhsinfo <- cbind(dhsinfo, stringr::str_split_fixed(dhsinfo$site_name, "_", 3))
#     names(dhsinfo) <- c('site_name','chr','bp1','bp2')
#     dhsinfo$chr <- gsub('chr','', dhsinfo$chr)
#     dhsinfo$bp1 <- as.numeric(as.character(dhsinfo$bp1))
#     dhsinfo$bp2 <- as.numeric(as.character(dhsinfo$bp2))
    
#     input_cds <- suppressWarnings(newCellDataSet(as(sc.data.subset, 'dgCMatrix'),
#                                 phenoData = methods::new('AnnotatedDataFrame', data = cellinfo),
#                                 featureData = methods::new('AnnotatedDataFrame', data = dhsinfo),
#                                 expressionFamily=negbinomial.size(),
#                                 lowerDetectionLimit=0))
#     input_cds@expressionFamily <- binomialff()
#     input_cds@expressionFamily@vfamily <- 'binomialff'
#     input_cds <- detectGenes(input_cds)
#     input_cds <- estimateSizeFactors(input_cds)
    
#     input_cds <- input_cds[fData(input_cds)$num_cells_expressed > 0,]
#     umap_coords <- sc.umap[colnames(sc.data.subset), c('UMAP1','UMAP2')]
#     colnames(umap_coords) <- NULL
    
#     cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = umap_coords, k=30)
#     window <- 1e6
#     data('human.hg19.genome')
#     distance_parameters <- estimate_distance_parameter(cicero_cds, window=window, maxit=100, sample_num=100, distance_constraint=500000, genomic_coords=human.hg19.genome)
#     mean_distance_parameter <- mean(unlist(distance_parameters))
#     cicero_out <- generate_cicero_models(cicero_cds, distance_parameter=mean_distance_parameter, window=window, genomic_coords=human.hg19.genome)
#     conns <- assemble_connections(cicero_out, silent=FALSE)
#     saveRDS(conns, file.path(wd, paste0('pbmc1_15.', cluster, '.1MB_cicero_conns.rds')))
#     write.table(conns, file.path(wd, paste0('pbmc1_15.', cluster, '.cicero_conns.txt')), sep='\t', quote=FALSE, row.names=FALSE)

# ## this step is to remove duplicated connections
# conns = conns[order(-conns$coaccess),]
# bed = cbind(str_split_fixed(conns[,1], "\\_", 3 ), str_split_fixed(conns[,2], "\\_", 3 ))
# ord = matrix(parRapply(clus, bed, function(x) x[order(x)] ), ncol=6, byrow=T)
# ord = cbind(ord[, c(5,1:2,5,3:4)], conns$coaccess)
# dedup = ord[!duplicated(ord),]
# dedup = data.frame( Peak1 = paste(dedup[,1], dedup[,2], dedup[,3], sep="_")  , 
#                     Peak2 = paste(dedup[,4], dedup[,5], dedup[,6], sep="_") , coaccess = dedup[,7]  )
              
              
#   write.table(dedup, file.path(wd, paste0('pbmc1_15.', cluster, '.cicero_conns_dedup.txt')), sep='\t', quote=FALSE, row.names=FALSE)

# }

In [43]:
for (cluster in clusters) {
   prefix = cluster
    sc.data.subset <- sc.data
    cellinfo <-data.frame(cells=colnames(sc.data.subset))
    row.names(cellinfo) <- cellinfo$cells
    dhsinfo <- data.frame(site_name=rownames(sc.data.subset))
    row.names(dhsinfo) <- dhsinfo$site_name
    dhsinfo <- cbind(dhsinfo, stringr::str_split_fixed(dhsinfo$site_name, "_", 3))
    names(dhsinfo) <- c('site_name','chr','bp1','bp2')
    dhsinfo$chr <- gsub('chr','', dhsinfo$chr)
    dhsinfo$bp1 <- as.numeric(as.character(dhsinfo$bp1))
    dhsinfo$bp2 <- as.numeric(as.character(dhsinfo$bp2))
    
    input_cds <- suppressWarnings(newCellDataSet(as(sc.data.subset, 'dgCMatrix'),
                                phenoData = methods::new('AnnotatedDataFrame', data = cellinfo),
                                featureData = methods::new('AnnotatedDataFrame', data = dhsinfo),
                                expressionFamily=negbinomial.size(),
                                lowerDetectionLimit=0))
    input_cds@expressionFamily <- binomialff()
    input_cds@expressionFamily@vfamily <- 'binomialff'
    input_cds <- detectGenes(input_cds)
    input_cds <- estimateSizeFactors(input_cds)
    
    input_cds <- input_cds[fData(input_cds)$num_cells_expressed > 0,]
    umap_coords <- sc.umap[colnames(sc.data.subset), c('UMAP1','UMAP2')]
    colnames(umap_coords) <- NULL
    
    cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = umap_coords, k=30)
    window <- 1e6
    data('human.hg19.genome')
    distance_parameters <- estimate_distance_parameter(cicero_cds, window=window, maxit=100, sample_num=100, distance_constraint=500000, genomic_coords=human.hg19.genome)
    mean_distance_parameter <- mean(unlist(distance_parameters))
    cicero_out <- generate_cicero_models(cicero_cds, distance_parameter=mean_distance_parameter, window=window, genomic_coords=human.hg19.genome)
    conns <- assemble_connections(cicero_out, silent=FALSE)
    #saveRDS(conns, file.path(wd, paste0('cicero/', cluster, '.1MB_cicero_conns.rds')))
    write.table(conns, file.path(wd, paste0('pbmc1_15.', cluster, '.cicero_conns2.txt')), sep='\t', quote=FALSE, row.names=FALSE)

## this step is to remove duplicated connections
conns = conns[order(-conns$coaccess),]
bed = cbind(str_split_fixed(conns[,1], "\\_", 3 ), str_split_fixed(conns[,2], "\\_", 3 ))
    #### mistake here because the order was not numeric ### must fix it
    
ord = matrix(parRapply(clus, bed, function(x) x[order(x)] ), ncol=6, byrow=T)
ord = cbind(ord[, c(5,1:2,5,3:4)], conns$coaccess)
dedup = ord[!duplicated(ord),]
dedup = data.frame( Peak1 = paste(dedup[,1], dedup[,2], dedup[,3], sep="_")  , 
                    Peak2 = paste(dedup[,4], dedup[,5], dedup[,6], sep="_") , coaccess = dedup[,7]  )
              
              
  write.table(dedup, file.path(wd, paste0('pbmc1_15.', cluster, '.cicero_conns_dedup2.txt')), sep='\t', quote=FALSE, row.names=FALSE)

}

Overlap QC metrics:
Cells per bin: 30
Maximum shared cells bin-bin: 26
Mean shared cells bin-bin: 0.120134359167468
Median shared cells bin-bin: 0



[1] "Successful cicero models:  5423"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                         194                          726 
[1] "Models with errors:  0"


In [ ]:
getwd()

In [18]:
lu = read.table("/nfs/lab/projects/lung_snATAC/analysis/cicero/lung.all_peaks.anno.bed")

In [20]:
table(lu$V4)


      Alveolar_type_1       Alveolar_type_2  Arterial_endothelial 
                90652                103999                 68040 
               B_cell Capillary_endothelial       CD34_fibroblast 
                41335                 62210                 72467 
             Ciliated Lymphatic_endothelial          Macrophage_1 
                77439                 42404                133635 
         Macrophage_2     Matrix_fibroblast         Myofibroblast 
                86296                 89876                 57882 
              NK_cell              Pericyte                T_cell 
                35904                 56726                 44441 

In [22]:
wd = '/nfs/lab/projects/lung_snATAC/analysis/cicero/'
setwd(wd)

In [23]:
zz   = gzfile('lung.merged_peaks.long_fmt.mtx.gz','rt')   ### long for matrix peak x barcode
data = read.table(zz,header=T,  sep="\t")